In [ ]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error


try:
    X_all = np.load('X_lstm.npy')
    y_all = np.load('y_lstm.npy')
    ids_all = np.load('ids_lstm.npy', allow_pickle=True)
except FileNotFoundError:
    print("Error: .npy files not found. Please run the preprocessing script first.")
    exit()

id_to_index = {pid: i for i, pid in enumerate(ids_all)}

fold_files = [f'fold_{i}.pkl' for i in range(5)]
all_fold_maes = []
all_fold_long_stay_maes = []

print(f"Loaded Global Data: Features {X_all.shape}, Targets {y_all.shape}")

def get_indices(original_pairs, lookup_map):
    valid_ids = [pair[1] for pair in original_pairs if pair[1] in lookup_map]
    return [lookup_map[pid] for pid in valid_ids]


for fold_num, file_path in enumerate(fold_files):
    try:
        with open(file_path, 'rb') as f:
            original_train, original_val, original_test = pickle.load(f)
    except FileNotFoundError:
        print(f"Skipping Fold {fold_num}: {file_path} not found.")
        continue

    train_idx = get_indices(original_train, id_to_index)
    val_idx   = get_indices(original_val, id_to_index)
    test_idx  = get_indices(original_test, id_to_index)

    X_train, y_train = X_all[train_idx], y_all[train_idx]
    X_val,   y_val   = X_all[val_idx],   y_all[val_idx]
    X_test,  y_test  = X_all[test_idx],  y_all[test_idx]
    test_ids = ids_all[test_idx]

    print(f"\n" + f" PROCESSING FOLD {fold_num} ".center(60, '#'))
    print(f"Sizes -> Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

  
    y_train_log = np.log1p(y_train)
    y_val_log   = np.log1p(y_val)

    if len(X_train) > 0 and len(X_val) > 0:
        

        weights_train = np.where(y_train > 7, 10.0, 1.0)
        
        tf.keras.backend.clear_session()
        
        model = Sequential([
            Masking(mask_value=0., input_shape=(X_train.shape[1], X_train.shape[2])),
            LSTM(128, return_sequences=True, activation='tanh'),
            Dropout(0.3),
            LSTM(64, return_sequences=False, activation='tanh'),
            Dropout(0.3),
            Dense(128, activation='relu'),
            Dense(64, activation='relu'),
            Dense(1, activation='linear') 
        ])

       
        model.compile(optimizer='adam', loss=tf.keras.losses.Huber(delta=1.0), metrics=['mae'])

        early_stop = EarlyStopping(
            monitor='val_loss', 
            patience=10, 
            restore_best_weights=True
        )
        
       
        model.fit(
            X_train, y_train_log,
            validation_data=(X_val, y_val_log),
            sample_weight=weights_train, 
            epochs=150,
            batch_size=128,
            callbacks=[early_stop],
            verbose=0
        )

   
        if len(X_test) > 0:
            log_preds = model.predict(X_test, verbose=0).flatten()
            
          
            real_preds = np.expm1(log_preds)
            real_preds = np.maximum(real_preds, 0)

            mae = mean_absolute_error(y_test, real_preds)
            all_fold_maes.append(mae)
            
            results = pd.DataFrame({
                'chemo_hadm_id': test_ids,
                'Actual': y_test,
                'Predicted': real_preds.round(2),
                'Error': np.abs(y_test - real_preds).round(2),
                'Residual': (real_preds - y_test).round(2)
            })

            print("-" * 40)
            print(f"Fold {fold_num} Global MAE: {mae:.4f}")
            
            long_stays = results[results['Actual'] > 7]
            if not long_stays.empty:
                l_mae = mean_absolute_error(long_stays['Actual'], long_stays['Predicted'])
                all_fold_long_stay_maes.append(l_mae)
                print(f"Fold {fold_num} Long-Stay MAE (>7d): {l_mae:.4f}")

            print("\n>>> TOP 10 WORST PREDICTIONS (Highest Error):")
            print(results.sort_values('Error', ascending=False).head(10).to_string(index=False))

            print("\n>>> TOP 10 BEST PREDICTIONS (Lowest Error):")
            print(results.sort_values('Error', ascending=True).head(10).to_string(index=False))

            bias = results['Residual'].mean()
            status = "OVERESTIMATING" if bias > 0 else "UNDERESTIMATING"
            print(f"\nAverage Bias: {bias:.4f} ({status})")

        else:
            print(f"Fold {fold_num}: No test samples found.")
    else:
        print(f"Fold {fold_num}: Insufficient data for training.")


if all_fold_maes:
    print("\n" + "="*60)
    print(f"FINAL CROSS-VALIDATION RESULTS".center(60))
    print("="*60)
    print(f"Average MAE (All Stays):  {np.mean(all_fold_maes):.4f}")
    if all_fold_long_stay_maes:
        print(f"Average MAE (Stays >7d): {np.mean(all_fold_long_stay_maes):.4f}")
    print("="*60)

Loaded Global Data: Features (1010, 14, 100), Targets (1010,)

#################### PROCESSING FOLD 0 #####################
Sizes -> Train: 733, Val: 79, Test: 198
----------------------------------------
Fold 0 Global MAE: 2.7390
Fold 0 Long-Stay MAE (>7d): 9.2483

>>> TOP 10 WORST PREDICTIONS (Highest Error):
 chemo_hadm_id  Actual  Predicted  Error  Residual
      22409075    41.0       0.97  40.03    -40.03
      21176832    41.0       6.25  34.75    -34.75
      27576498    25.0       5.46  19.54    -19.54
      24952654    19.0       0.78  18.22    -18.22
      25841662    24.0       9.00  15.00    -15.00
      23344242    15.0       2.72  12.28    -12.28
      22984596    18.0       5.78  12.22    -12.22
      27253820    16.0       5.22  10.78    -10.78
      22994573    12.0       1.89  10.11    -10.11
      28310302     2.0      11.98   9.98      9.98

>>> TOP 10 BEST PREDICTIONS (Lowest Error):
 chemo_hadm_id  Actual  Predicted  Error  Residual
      26617869     1.0       1